<a href="https://colab.research.google.com/github/Pushpalatha-H/ML/blob/main/17_Hyper_parameter_tuning(GridSearchCV).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
# load iris
from sklearn import datasets
iris = datasets.load_iris()

In [3]:
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [4]:
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [5]:
df['target'] = iris.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
df['flower_name'] = df['target'].apply(lambda x: iris.target_names[x])
df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,flower_name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target)

**using SVM**

In [8]:
# svm
from sklearn import svm
model = svm.SVC(kernel='rbf', C=30, gamma='auto')
model.fit(x_train,y_train)

SVC(C=30, gamma='auto')

In [9]:
model.score(x_test, y_test)

0.9473684210526315

**Using CrossValScore**

In [10]:
from sklearn.model_selection import cross_val_score

In [11]:
cross_val_score(svm.SVC(kernel='rbf', C=10, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [12]:
cross_val_score(svm.SVC(kernel='linear', C=10, gamma='auto'), iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [13]:
cross_val_score(svm.SVC(kernel='rbf', C=20, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

**writing loop for cross_val_score**

In [14]:
# writing in the form of loop
import numpy as np
avg_scores = {}
kernel = ['rbf', 'linear']
C = [1, 5, 10]
for k_val in kernel:
  for C_val in C:
    cv_scores = cross_val_score(svm.SVC(kernel = k_val, C = C_val, gamma='auto'), iris.data, iris.target)
    avg_scores[k_val + '_' + str(C_val)] =np.average(cv_scores)
avg_scores


{'rbf_1': 0.9800000000000001,
 'rbf_5': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'linear_1': 0.9800000000000001,
 'linear_5': 0.9800000000000001,
 'linear_10': 0.9733333333333334}

**Using GridSearchCV**

In [15]:
# GridSearchCV
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'),{
    'C' : [1, 10, 20],
    'kernel' : ['rbf', 'linear']
},cv=5, return_train_score= False)
clf.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20], 'kernel': ['rbf', 'linear']})

In [16]:
clf.score(x_test, y_test)

0.9473684210526315

In [17]:
clf.cv_results_

{'mean_fit_time': array([0.0021142 , 0.00187831, 0.00265608, 0.00164294, 0.00216422,
        0.00255852]),
 'std_fit_time': array([0.00060251, 0.00031164, 0.00077397, 0.00018309, 0.00078114,
        0.00103629]),
 'mean_score_time': array([0.00141139, 0.00120997, 0.00142026, 0.00190797, 0.00183358,
        0.00129375]),
 'std_score_time': array([0.00057052, 0.00027679, 0.00059947, 0.00104413, 0.00101874,
        0.00028867]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [18]:
df_clf = pd.DataFrame(clf.cv_results_)
df_clf.head(3)[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.973123
1,1,linear,1.000000
2,10,rbf,0.973123


**Using RandomizedSearchCV**

In [19]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
    'kernel':['rbf', 'linear'],
    'C' : [1, 5, 10]
}, cv=5, return_train_score = False)
rs.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'),
                   param_distributions={'C': [1, 5, 10],
                                        'kernel': ['rbf', 'linear']})

In [20]:
rs.score(x_test, y_test)

0.9473684210526315

In [21]:
rs_df= pd.DataFrame(rs.cv_results_)
rs_df.head(3)[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.973123
1,1,linear,1.000000
2,5,rbf,0.982213


**Using GridSearchCV to find which model and parameter works good by using GridSearch CV**

In [22]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'SVM': {
        'model': svm.SVC(gamma='auto'),
        'params':{
            'C' : [1, 5, 10],
            'kernel' : ['rbf', 'linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params':{
            'n_estimators' : [1, 5, 10]
        }
    },
    'logistic_regression': {
        'model' : LogisticRegression(solver='liblinear', multi_class = 'auto'),
        'params' : {
            'C': [ 1, 5, 10]
        }
    }
}

In [23]:
scores = []
for model_name, mp in model_params.items():
  clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score = False)
  clf.fit(iris.data, iris.target)
  scores.append({
      'model' : model_name,
      'best_scores' : clf.best_score_,
      'best_params' : clf.best_params_
  })
scores

[{'model': 'SVM',
  'best_scores': 0.9800000000000001,
  'best_params': {'C': 1, 'kernel': 'rbf'}},
 {'model': 'random_forest',
  'best_scores': 0.9666666666666668,
  'best_params': {'n_estimators': 10}},
 {'model': 'logistic_regression',
  'best_scores': 0.9666666666666668,
  'best_params': {'C': 5}}]

In [24]:
scores = pd.DataFrame(scores)
scores

,model,best_scores,best_params
0,SVM,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}
